<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

{0: bike, 1: chair, 2: camel, 3: rabbit, 4: lamp, 5: clock,
6: tree?, 7: fox, 8: hedgehog, 9: bed, 10: child,
11: Dinosaur, 12: building, 13: tree?, 14: apple, 15: animal?,
16: animal?, 17: kangaroo, 18: tree?, 19: snake, 20: beetle,
21: cupboard, 22: mouse, 23: tree?, 24: house, 25: Wolf}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [ ]:
# search file paths
from glob import glob


img_paths_train = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png')) # *: all
img_paths_val = sorted(glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png'))

In [ ]:
# number of images
len(img_paths_train), len(img_paths_val)

(22500, 2500)

In [ ]:
import torch
import cv2


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, img_size):
        self.img_paths = img_paths
        self.img_size = img_size

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = cv2.imread(path) # read img
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB
        img = cv2.resize(img, (self.img_size, self.img_size)) # Resize iamge
        img = img / 255.0 # 0~255 -> 0.~1.

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])

        # transform img to tensor
        img = torch.tensor(img, dtype=torch.float)
        img = img.permute(2, 0, 1) # (H, W, C) -> (C, H, W)
        cls_idx = torch.tensor(cls_idx, dtype=torch.long)

        return img, cls_idx

In [ ]:
# Build dataset
IMG_SIZE = 32
train_ds = MyDataset(img_paths_train, IMG_SIZE)
val_ds = MyDataset(img_paths_val, IMG_SIZE)

In [ ]:
# Build DataLoaders
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE)

In [ ]:
# Define Model class
from torch import nn
import torchvision
from torchsummary import summary # model summary


class CNN_scratch(nn.Module):
    def __init__(self, kernel_size=3):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=kernel_size, padding='same'),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size, padding='same'),
            nn.BatchNorm2d(num_features=32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2), # img_size/2

            nn.Conv2d(32, 64, kernel_size, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size, padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2), # img_size/4

            nn.Conv2d(64, 128, kernel_size, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2), # img_size/8
        )
        self.flatten = nn.Flatten() # (128, 4, 4) -> (128*4*4)
        self.classifier = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(128*4*4, 256),
            nn.Dropout(0.3),
            nn.ReLU(),
            nn.Linear(256, 50),
        )
        # self.flatten = nn.Sequential(
        #     nn.AdaptiveAvgPool2d(1),
        #     nn.Flatten() # (128, 1, 1)
        # )
        # self.classifier = nn.Sequential(
        #     nn.Linear(128, 50),
        # )
    def forward(self, x):
        x = self.feature_extractor(x) # img to feature maps
        x = self.flatten(x) # feature maps -> feature vectors
        x = self.classifier(x) # classification
        return x

In [ ]:
# Build a model and move to GPU device
model = CNN_scratch(kernel_size=3).to(device)

In [ ]:
print(model)

CNN_scratch(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=

In [ ]:
# Model summary
summary(model, (3, IMG_SIZE, IMG_SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 32, 32]             896
       BatchNorm2d-2           [-1, 32, 32, 32]              64
              ReLU-3           [-1, 32, 32, 32]               0
            Conv2d-4           [-1, 32, 32, 32]           9,248
       BatchNorm2d-5           [-1, 32, 32, 32]              64
              ReLU-6           [-1, 32, 32, 32]               0
         MaxPool2d-7           [-1, 32, 16, 16]               0
            Conv2d-8           [-1, 64, 16, 16]          18,496
       BatchNorm2d-9           [-1, 64, 16, 16]             128
             ReLU-10           [-1, 64, 16, 16]               0
           Conv2d-11           [-1, 64, 16, 16]          36,928
      BatchNorm2d-12           [-1, 64, 16, 16]             128
             ReLU-13           [-1, 64, 16, 16]               0
        MaxPool2d-14             [-1, 6

#### Training

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

In [ ]:
from tqdm.auto import tqdm # (optional) progress bar


def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode.
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device), y.to(device) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [ ]:
import numpy as np


EPOCHS = 100
logs = {
    'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
}

# Earlystopping
patience = 5
counter = 0
best_loss = np.inf

for epoch in tqdm(range(EPOCHS)):
    train_loss, train_acc = train(train_loader, model, loss_fn, optimizer)
    val_loss, val_acc = test(val_loader, model, loss_fn)

    print(f'EPOCH: {epoch:04d} \
    train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
    val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

    logs['train_loss'].append(train_loss)
    logs['train_acc'].append(train_acc)
    logs['val_loss'].append(val_loss)
    logs['val_acc'].append(val_acc)

    torch.save(model.state_dict(), "last.pth")
    # chcek improvement
    if val_loss < best_loss:
        counter = 0
        best_loss = val_loss
        torch.save(model.state_dict(), "best.pth")
    else:
        counter += 1
    if counter >= patience:
        print("Earlystop!")
        break

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/352 [00:00<?, ?it/s]

EPOCH: 0000     train_loss: 3.4742, train_acc: 0.121     val_loss: 3.1096, val_acc: 0.201 


  0%|          | 0/352 [00:00<?, ?it/s]

EPOCH: 0001     train_loss: 2.9281, train_acc: 0.224     val_loss: 2.6719, val_acc: 0.282 


...

EPOCH: 0024     train_loss: 0.7784, train_acc: 0.753     val_loss: 1.6353, val_acc: 0.545 


  0%|          | 0/352 [00:00<?, ?it/s]

EPOCH: 0025     train_loss: 0.7368, train_acc: 0.765     val_loss: 1.7068, val_acc: 0.538 


  0%|          | 0/352 [00:00<?, ?it/s]

EPOCH: 0026     train_loss: 0.6952, train_acc: 0.780     val_loss: 1.6513, val_acc: 0.559 
Earlystop!


#### Evaluation

In [ ]:
model = model.to(device) # move model to GPU
model.eval() # Set model to evaluation mode
# Create temp tensor
y_true = torch.tensor([], dtype=torch.long).to(device)
y_pred_logits = torch.tensor([]).to(device)

# deactivate autograd engine and reduce memory usage and speed up computations
with torch.no_grad():
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)

        pred = model(x)
        # Concatenate Ground truth and prediction
        y_true = torch.cat((y_true, y), dim=0)
        y_pred_logits = torch.cat((y_pred_logits, pred), dim=0)

y_pred_cls = y_pred_logits.argmax(1) # get class idx with max prob (100, 3) -> (100, )
y_pred_probs = nn.Softmax(dim=1)(y_pred_logits) # convert logits to probability

# Move tensor to CPU and convert to np.ndarray
y_true = y_true.cpu().numpy()
y_pred_cls = y_pred_cls.cpu().numpy()
y_pred_probs = y_pred_probs.cpu().numpy()

In [ ]:
# Shape
y_true.shape, y_pred_probs.shape, y_pred_cls.shape

((2500,), (2500, 50), (2500,))

In [ ]:
# calculate ACC
acc = sum(y_true == y_pred_cls) / len(y_true)
acc

0.5588